In [2]:
import argparse
import math
from intern.remote.boss import BossRemote
from intern.resource.boss.resource import *
import configparser
#import grequests # for async requests, conflicts with requests somehow
import requests
import numpy as np
from numpy import genfromtxt
import h5py
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import shutil
import blosc
from IPython.core.debugger import set_trace
import sys
import os
import itertools
from functools import partial
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
from multiprocessing import cpu_count
import csv
import datetime
import morton

## JLP user python library
#import toolbox

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [18]:
def Annuli3dMask1(x,y,z):
    def f(a,b,c):
        w = abs(a) + abs(b) + abs(c)
        return(w) 
    
    mask = np.array([0 for i in range(x*y*z)], dtype = np.float32).reshape(x,y,z)
    for i in range(x):
        for j in range(y):
            for k in range(z):
                mi = f(i - np.floor(x/2), j - np.floor(y/2), k - np.floor(z/2))
                if mi == 0:
                    mask[i,j,k] = 1
                else:
                    mask[i,j,k] = mi
                    
    return(mask)

def Annuli3dMaskInf(x,y,z):
    def f(a,b,c):
        w = max(abs(a),abs(b),abs(c))
        return(w) 
    
    mask = np.array([0 for i in range(x*y*z)], dtype = np.float32).reshape(x,y,z)
    for i in range(x):
        for j in range(y):
            for k in range(z):
                mi = f(i - np.floor(x/2), j - np.floor(y/2), k - np.floor(z/2))
                if mi == 0:
                    mask[i,j,k] = 1
                else:
                    mask[i,j,k] = mi
                    
    return(mask)

#gm0 = toolbox.gaussian3dMask([0,0,0], [0.4,0.4,0.4], 11,11,11, [5,5,5])

M1 = Annuli3dMask1(11,11,11)
nu = np.unique(M1, return_counts = True)
nu[0]/(nu[1]/np.prod(np.shape(M1)))

M1

array([[[15., 14., 13., ..., 13., 14., 15.],
        [14., 13., 12., ..., 12., 13., 14.],
        [13., 12., 11., ..., 11., 12., 13.],
        ...,
        [13., 12., 11., ..., 11., 12., 13.],
        [14., 13., 12., ..., 12., 13., 14.],
        [15., 14., 13., ..., 13., 14., 15.]],

       [[14., 13., 12., ..., 12., 13., 14.],
        [13., 12., 11., ..., 11., 12., 13.],
        [12., 11., 10., ..., 10., 11., 12.],
        ...,
        [12., 11., 10., ..., 10., 11., 12.],
        [13., 12., 11., ..., 11., 12., 13.],
        [14., 13., 12., ..., 12., 13., 14.]],

       [[13., 12., 11., ..., 11., 12., 13.],
        [12., 11., 10., ..., 10., 11., 12.],
        [11., 10.,  9., ...,  9., 10., 11.],
        ...,
        [11., 10.,  9., ...,  9., 10., 11.],
        [12., 11., 10., ..., 10., 11., 12.],
        [13., 12., 11., ..., 11., 12., 13.]],

       ...,

       [[13., 12., 11., ..., 11., 12., 13.],
        [12., 11., 10., ..., 10., 11., 12.],
        [11., 10.,  9., ...,  9., 10., 11.

In [19]:
test = np.ones(1331).reshape(11,11,11)
c = np.unique(M1, return_counts = True)[1]
u = np.unique(M1)
for i in range(len(u)):
    print(np.sum(test[M1 == u[i]])/c[i])

print(u)
print(c)


1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.]
[  7  18  38  66 102 140 168 180 176 156 120  80  48  24   8]


In [20]:
def F0(cubes):
    f0 = [[np.sum(cubes[i,j,:,:,:]) for j in range(cubes.shape[1])] for i in range(cubes.shape[0])]
    #f0 = np.sum(cube)
    return(f0)
    
def F1(cubes):
    s = cubes.shape

    f1 = [[np.sum(cubes[i,j,:,:,:]) for j in range(cubes.shape[1])] for i in range(cubes.shape[0])]
    out = f1

    return(out)



In [21]:
def getCube(di):
    data = di['rem'].get_cutout(di['ch_rsc'], di['res'], di['xrng'],
          di['yrng'] ,di['zrng'])
    print(di['loc']) #DEBUG
    sys.stdout.flush() #DEBUG
    ### OUTPUT will be a numpy array in Z Y X order 
    #avg = np.mean(data)
    out = [np.sum(data[M1 == u[i]])/c[i] for i in range(len(u))]
    return(out)

def getWCube(di):
    data = di['rem'].get_cutout(di['ch_rsc'], di['res'], di['xrng'],
          di['yrng'] ,di['zrng'])
    print(di['loc']) #DEBUG
    sys.stdout.flush() #DEBUG
    ### OUTPUT will be a numpy array in Z Y X order 
    #avg = np.mean(data)
    wdata = data / M1
    out = [np.sum(wdata[M1 == u[i]])/c[i] for i in range(len(u))]
    return(out)


In [22]:
CONFIG_FILE = "neurodata.conf"
COLL_NAME = "collman"
EXP_NAME = "M247514_Rorb_1_Site3Align2_LENS_Session1_CROP"
COORD_FRAME = COLL_NAME + "_" + EXP_NAME
BF = 5

#LOCATIONS = "rorb_locations_at_buffered.csv"
LOCATIONS = "GoogleDocData.csv"


CHAN_NAMES = ['DAPI1', 'DAPI2', 'DAPI3', 'GABA', 'GAD2', 'Gephyrin', 'GluN1',
                'MBP', 'PSD95', 'synapsin', 'TdTomato', 'VGlut1']

num_threads = 8

In [23]:
config = configparser.ConfigParser()
config.read(CONFIG_FILE)
TOKEN = config['Default']['token']
boss_url = ''.join( ( config['Default']['protocol'],'://',config['Default']['host'],'/v1/' ) )
#print(boss_url)
#'https://api.boss.neurodata.io/v1/'

#intern
rem = BossRemote(CONFIG_FILE)

In [24]:
L = genfromtxt(LOCATIONS,  delimiter=',', skip_header = 1, dtype='int32').tolist()
#m = morton.Morton(dimensions=3, bits=64)
#Lzo = [m.pack(L[i][0], L[i][1], L[i][2]) for i in range(len(L))]
#Lzo = sorted([m.pack(L[i][0], L[i][1], L[i][2]) for i in range(len(L))])
#                                                                                               
loc = L

x_rng = [[x[0]-BF, x[0]+BF+1] for x in loc] 
y_rng = [[y[1]-BF, y[1]+BF+1] for y in loc] 
z_rng = [[z[2]-BF, z[2]+BF+1] for z in loc]
ids   = [i[3] for i in loc]
loc

[[217, 204, 30, 0, 0, 0, 0, 0],
 [234, 180, 42, 4, 0, 1, 1, 0],
 [240, 241, 10, 5, 0, 1, 1, 0],
 [201, 211, 12, 6, 0, 0, -1, 1],
 [249, 221, 11, 7, 0, 0, 0, 0],
 [256, 218, 18, 8, 0, 0, 0, 0],
 [228, 183, 39, 9, 0, 0, 1, 0],
 [222, 181, 41, 10, 0, 0, 2, 0],
 [221, 184, 34, 11, 0, 0, 1, 0],
 [187, 192, 34, 12, 0, 0, 0, 0],
 [191, 190, 32, 13, 0, 0, 0, 0],
 [200, 193, 34, 14, 0, 0, 0, 0],
 [203, 194, 23, 15, 0, 0, 0, 0],
 [206, 179, 20, 16, 0, 1, 0, 0],
 [205, 188, 16, 17, 0, 0, 0, 0],
 [209, 188, 20, 18, 0, 0, 0, 0],
 [197, 194, 20, 19, 1, 0, 0, 0],
 [197, 194, 18, 20, 0, 0, 0, 0],
 [194, 182, 18, 21, 0, 1, 0, 0],
 [200, 187, 22, 22, 0, 0, 0, 0],
 [222, 183, 24, 23, 0, 0, 2, 0],
 [230, 177, 33, 24, 0, 0, 0, 0],
 [225, 172, 36, 25, 0, 1, 0, 0],
 [222, 163, 38, 26, 0, 1, 0, 0],
 [207, 164, 41, 27, 0, 1, 0, 0],
 [201, 163, 19, 29, 0, 0, 0, 0],
 [208, 163, 34, 31, 0, 1, 1, 0],
 [210, 167, 6, 32, 1, 1, 0, 0],
 [217, 189, 14, 34, 0, 0, 1, 0],
 [231, 195, 10, 35, 0, 0, 0, 0],
 [225, 178, 14, 3

In [26]:
ChanList = []
for ch in CHAN_NAMES:
    di = [{
          'rem': rem,
          'ch_rsc':
            ChannelResource(ch,COLL_NAME,EXP_NAME,'image',datatype='uint8'),
          'ch'  : ch,
          'res' : 0,
          'loc' : loc[i],
          'xrng': x_rng[i],  
          'yrng': y_rng[i],  
          'zrng': z_rng[i],  
          'bf'  : BF 
         } for i in range(len(loc))]

    with ThreadPool(num_threads) as tp:
        out = tp.map(getWCube, di)
        print(ch) ##DEBUG
        sys.stdout.flush() #DEBUG

    ChanList.append(np.asarray(out))

outArray = np.hstack(ChanList)
## outArray will be a numpy array in [channel, synapse, z, y, x] order
## this is C-ordering


[198, 216, 40, 131, 0, 1, 0, 0]
[208, 163, 34, 31, 0, 1, 1, 0]
[193, 188, 12, 64, 0, 0, 0, 0]
[244, 184, 41, 170, 0, 1, 1, 0]
[236, 188, 15, 203, 0, 0, 0, 0]
[197, 157, 16, 97, 0, 1, 1, 0]
[242, 261, 27, 234, 0, 1, 0, 0]
[217, 204, 30, 0, 0, 0, 0, 0]
[195, 226, 8, 136, 1, 1, 2, 0]
[210, 167, 6, 32, 1, 1, 0, 0]
[211, 169, 21, 172, 0, 1, 1, 0]
[204, 182, 11, 65, 0, 0, 0, 0]
[195, 156, 36, 98, 0, 0, -1, 1]
[237, 191, 13, 204, 1, 0, 0, 0]
[240, 254, 35, 235, 0, 0, 0, 0]
[234, 180, 42, 4, 0, 1, 1, 0]
[189, 222, 6, 137, 0, 0, 1, 0]
[190, 183, 43, 173, 0, 0, 0, 0]
[217, 189, 14, 34, 0, 0, 1, 0]
[206, 174, 14, 66, 0, 0, 0, 0]
[194, 158, 34, 99, 0, 0, 3, 0]
[234, 240, 40, 236, 0, 1, 0, 0][240, 241, 10, 5, 0, 1, 1, 0]

[235, 189, 12, 205, 0, 0, 1, 0]
[206, 216, 24, 138, 0, 1, 0, 0]
[231, 195, 10, 35, 0, 0, 0, 0]
[221, 154, 40, 207, 0, 0, 1, 0]
[205, 161, 32, 67, 0, 1, -1, 1]
[230, 191, 30, 174, 0, 1, 0, 0]
[198, 154, 38, 101, 0, 0, 0, 0]
[210, 238, 36, 237, 1, 1, 0, 0]
[201, 211, 12, 6, 0, 0, -1

[284, 214, 41, 473, 1, 1, -1, 1]
[228, 193, 5, 307, 0, 0, 0, 0]
[309, 217, 36, 504, 0, 0, 0, 0]
[260, 247, 12, 345, 0, 1, 0, 0]
[296, 183, 26, 444, 0, 0, 2, 0]
[196, 241, 21, 275, 0, 0, 2, 0]
[306, 176, 33, 413, 1, 0, 0, 0]
[286, 165, 7, 381, 0, 0, 0, 0]
[283, 196, 10, 309, 0, 1, -1, 1]
[279, 206, 14, 474, 0, 0, 0, 0]
[313, 193, 22, 445, 0, 0, 1, 0]
[310, 218, 33, 505, 0, 0, 0, 0]
[206, 245, 14, 276, 0, 0, 0, 0]
[259, 250, 31, 346, 1, 1, 0, 0]
[305, 163, 20, 383, 0, 1, 0, 0]
[309, 175, 35, 414, 0, 0, 2, 0]
[301, 204, 19, 475, 1, 0, 0, 0]
[261, 195, 14, 311, 0, 0, 2, 0]
[216, 237, 22, 279, 0, 0, 1, 0]
[308, 185, 16, 446, 0, 1, 0, 0]
[311, 216, 21, 506, 0, 1, 0, 0]
[287, 188, 42, 415, 0, 0, 0, 0]
[271, 247, 32, 347, 0, 0, 3, 0]
[301, 163, 32, 384, 0, 1, 1, 0]
[291, 209, 8, 476, 0, 0, 1, 0]
[210, 231, 23, 280, 0, 0, 0, 0]
[301, 216, 11, 507, 0, 0, 0, 0]
[308, 193, 11, 447, 1, 0, 0, 0]
[268, 201, 14, 312, 0, 0, 1, 0]
[265, 250, 38, 348, 0, 0, 0, 0]
[299, 183, 5, 385, 0, 0, -1, 1]
[300, 205

[284, 244, 9, 586, 0, 0, 1, 0]
[309, 254, 16, 558, 1, 0, 2, 0]
[255, 278, 37, 679, 0, 0, 0, 0]
[251, 165, 20, 820, 0, 0, 0, 0]
[286, 275, 21, 647, 1, 1, 0, 0]
[210, 284, 37, 711, 0, 1, 1, 0]
[256, 283, 22, 738, 0, 1, -1, 1]
[274, 258, 9, 619, 0, 0, 0, 0]
[300, 245, 15, 587, 0, 0, 0, 0]
[248, 268, 40, 680, 0, 0, 0, 0]
[302, 261, 42, 559, 0, 0, 0, 0]
[254, 154, 38, 821, 0, 0, 1, 0]
[288, 271, 12, 648, 1, 0, 0, 0]
[255, 283, 27, 739, 0, 0, 0, 0]
[258, 262, 42, 681, 0, 0, 0, 0]
[214, 282, 18, 712, 0, 0, 0, 0]
[275, 261, 7, 620, 0, 0, 0, 0]
[281, 249, 42, 562, 0, 0, 0, 0]
[290, 255, 14, 588, 0, 0, 0, 0]
[252, 171, 32, 822, 0, 0, 0, 0]
[296, 266, 14, 649, 0, 0, 0, 0]
[244, 279, 24, 740, 0, 1, 1, 0]
[257, 283, 38, 686, 0, 0, -1, 1]
[267, 263, 21, 621, 0, 1, 0, 0]
[219, 286, 8, 713, 1, 1, 0, 0]
[276, 245, 37, 563, 0, 0, 0, 0]
[231, 282, 24, 741, 0, 0, 0, 0]
[256, 186, 18, 823, 0, 0, 0, 0]
[287, 252, 14, 589, 0, 0, 2, 0]
[293, 263, 22, 650, 0, 0, 0, 0]
[257, 282, 42, 687, 0, 1, -1, 1]
[280, 258

[255, 249, 10, 1016, 0, 1, 1, 0]
[312, 189, 10, 894, 0, 1, -1, 1]
[262, 186, 18, 862, 0, 0, 0, 0]
[200, 186, 16, 984, 1, 0, 0, 0]
[191, 274, 30, 923, 0, 0, 0, 0]
[203, 269, 17, 1058, 0, 0, 2, 0]
[254, 224, 24, 955, 0, 1, 0, 0]
[257, 270, 30, 1017, 0, 0, -1, 1]
[262, 177, 8, 863, 0, 1, 0, 0]
[204, 193, 36, 985, 0, 1, -1, 1]
[200, 269, 22, 924, 1, 0, 0, 0][276, 190, 36, 895, 0, 0, 2, 0]

[231, 238, 30, 1059, 0, 0, 0, 0]
[251, 220, 24, 956, 0, 1, 0, 0]
[200, 252, 28, 1019, 0, 0, 0, 0]
[263, 197, 9, 864, 0, 1, -1, 1]
[200, 189, 42, 986, 0, 1, -1, 1]
[261, 191, 40, 896, 1, 1, 0, 0]
[246, 248, 32, 925, 1, 1, 2, 0]
[188, 246, 40, 1060, 0, 0, 2, 0]
[250, 215, 24, 957, 0, 1, 1, 0]
[211, 249, 41, 1020, 0, 0, -1, 1]
[289, 195, 12, 865, 0, 1, -1, 1]
[207, 186, 10, 987, 0, 0, -1, 1]
[267, 255, 16, 927, 0, 1, 0, 0]
[274, 194, 28, 897, 0, 0, 1, 0]
[200, 240, 42, 1061, 0, 1, 0, 0]
[247, 206, 25, 958, 0, 0, 0, 0]
[208, 249, 42, 1021, 0, 0, -1, 1]
[275, 186, 6, 867, 0, 0, 0, 0]
[227, 171, 30, 990, 0, 0,

[300, 174, 28, 404, 0, 1, 1, 0]
[201, 163, 19, 29, 0, 0, 0, 0]
[312, 190, 28, 437, 0, 0, 0, 0]
[266, 221, 29, 335, 0, 0, 3, 0]
[219, 235, 27, 269, 0, 1, 0, 0]
[227, 215, 12, 299, 1, 1, 2, 0]
[298, 160, 24, 374, 0, 0, 0, 0]
[317, 189, 11, 466, 0, 0, 1, 0]
[287, 204, 6, 499, 0, 0, 2, 0]
[312, 198, 26, 439, 0, 0, -1, 1]
[295, 172, 36, 405, 0, 0, 0, 0]
[231, 214, 18, 300, 1, 1, 2, 0]
[198, 236, 28, 270, 1, 0, 0, 0]
[266, 218, 41, 336, 0, 0, 0, 0]
[309, 192, 7, 467, 0, 0, 3, 0]
[296, 212, 8, 500, 0, 0, 0, 0]
[307, 196, 28, 440, 0, 1, 1, 0]
[303, 179, 33, 406, 1, 1, 0, 0]
[293, 158, 16, 375, 0, 1, 0, 0]
[243, 210, 43, 301, 0, 0, 1, 0]
[278, 217, 18, 339, 0, 1, -1, 1]
[192, 244, 16, 271, 1, 1, 0, 0]
[301, 209, 6, 502, 0, 0, 0, 0]
[298, 200, 12, 471, 0, 1, 0, 0]
[302, 194, 22, 441, 0, 0, 1, 0]
[315, 157, 10, 376, 0, 0, 0, 0]
[236, 199, 24, 304, 1, 1, 0, 0]
[300, 178, 36, 407, 1, 1, 0, 0]
[229, 248, 40, 272, 0, 0, 1, 0]
[314, 210, 24, 503, 0, 0, 0, 0]
[285, 211, 10, 472, 0, 1, -1, 1]
[257, 236,

...

[271, 274, 10, 693, 0, 1, 0, 0]
[221, 267, 28, 723, 0, 1, -1, 1]
[264, 250, 8, 753, 0, 0, 2, 0]
[297, 263, 26, 632, 0, 1, -1, 1]
[309, 262, 29, 595, 0, 0, 0, 0]
[275, 252, 10, 566, 0, 1, 0, 0][272, 260, 37, 660, 0, 0, 0, 0]

[257, 162, 40, 831, 1, 0, 0, 0]
[285, 258, 7, 761, 0, 0, 0, 0]
[271, 278, 8, 694, 0, 1, 0, 0]
[223, 262, 28, 724, 0, 0, 2, 0]
[275, 251, 39, 567, 0, 0, 0, 0]
[292, 263, 28, 633, 0, 0, -1, 1]
[279, 175, 38, 833, 0, 0, -1, 1]
[267, 254, 6, 768, 0, 0, 0, 0][272, 266, 29, 661, 0, 0, 0, 0]

[317, 264, 20, 596, 0, 0, 0, 0][263, 284, 12, 695, 0, 0, 0, 0]

[225, 270, 30, 725, 0, 0, 0, 0]
[300, 266, 26, 634, 0, 0, 2, 0]
[277, 251, 37, 568, 0, 0, 0, 0]
[276, 184, 37, 834, 0, 1, -1, 1]
[232, 284, 6, 776, 0, 0, 1, 0]
[269, 265, 28, 662, 0, 0, 0, 0]
[257, 283, 6, 696, 0, 1, 0, 0]
[249, 284, 31, 726, 0, 0, 0, 0]
[309, 263, 20, 597, 0, 0, 0, 0]
[273, 195, 40, 835, 0, 0, -1, 1]
[286, 254, 25, 569, 0, 1, 0, 0]
[256, 160, 34, 840, 0, 0, 1, 0]
[279, 180, 22, 807, 0, 0, -1, 1]
[286, 1

[231, 195, 10, 35, 0, 0, 0, 0]
[210, 238, 36, 237, 1, 1, 0, 0]
[205, 216, 12, 139, 0, 0, 2, 0]
[194, 154, 38, 102, 0, 1, 0, 0]
[231, 195, 25, 175, 1, 1, 0, 0]
[217, 265, 40, 209, 0, 0, 0, 0]
[249, 221, 11, 7, 0, 0, 0, 0][225, 178, 14, 36, 0, 0, 0, 0]

[188, 255, 32, 238, 0, 0, 0, 0]
[192, 224, 21, 140, 0, 0, 0, 0]
[227, 190, 26, 176, 0, 0, 0, 0]
[188, 155, 40, 103, 0, 1, -1, 1]
[196, 274, 38, 210, 0, 0, 1, 0]
[256, 218, 18, 8, 0, 0, 0, 0]
[211, 263, 27, 239, 0, 1, 0, 0]
[227, 171, 18, 37, 0, 0, 0, 0][194, 171, 14, 68, 0, 0, 0, 0]

[192, 205, 36, 142, 0, 0, 1, 0]
[226, 198, 36, 177, 0, 0, 0, 0]
[192, 162, 7, 106, 0, 0, 1, 0]
[209, 274, 33, 211, 0, 0, 0, 0]
[228, 183, 39, 9, 0, 0, 1, 0]
[229, 271, 23, 240, 0, 0, 0, 0][193, 160, 42, 69, 0, 1, 0, 0]

[234, 174, 20, 38, 0, 0, 1, 0]
[197, 213, 24, 143, 1, 1, 2, 0]
[191, 156, 5, 107, 0, 1, 0, 0][228, 186, 30, 178, 0, 0, 0, 0]

[244, 262, 38, 212, 0, 0, 2, 0]
[222, 181, 41, 10, 0, 0, 2, 0]
[220, 271, 21, 241, 0, 1, 0, 0]
[188, 166, 12, 70, 0, 

[301, 163, 32, 384, 0, 1, 1, 0]
[281, 187, 16, 416, 0, 0, 0, 0]
[268, 201, 14, 312, 0, 0, 1, 0]
[300, 187, 12, 448, 0, 0, 0, 0]
[210, 233, 8, 282, 0, 0, 0, 0]
[300, 205, 16, 477, 0, 0, 0, 0]
[311, 219, 10, 510, 1, 1, 0, 0]
[299, 183, 5, 385, 0, 0, -1, 1]
[265, 250, 38, 348, 0, 0, 0, 0][290, 187, 16, 417, 0, 0, 0, 0]

[259, 202, 32, 313, 0, 1, 1, 0]
[300, 191, 8, 449, 0, 0, 0, 0]
[207, 225, 38, 284, 0, 0, 0, 0]
[300, 203, 38, 478, 0, 1, 0, 0]
[315, 221, 30, 511, 0, 0, 2, 0]
[282, 171, 34, 386, 0, 1, 0, 0]
[259, 205, 41, 316, 0, 0, -1, 1]
[266, 237, 43, 350, 0, 0, -1, 1]
[276, 199, 9, 451, 0, 0, 2, 0]
[279, 192, 18, 419, 0, 0, 0, 0][239, 218, 34, 285, 1, 1, 0, 0]

[292, 204, 40, 479, 0, 0, 0, 0]
[317, 219, 26, 512, 0, 0, 0, 0]
[282, 175, 22, 387, 0, 0, 0, 0]
[274, 243, 37, 353, 0, 0, 2, 0]
[267, 204, 40, 318, 0, 1, 1, 0]
[296, 198, 22, 452, 0, 1, 0, 0]
[281, 181, 28, 420, 0, 0, 0, 0]
[195, 232, 34, 286, 0, 0, 0, 0]
[286, 212, 31, 481, 0, 1, 0, 0]
[310, 223, 37, 513, 0, 0, 0, 0]
[282, 166

[287, 252, 14, 589, 0, 0, 2, 0]
[293, 263, 22, 650, 0, 0, 0, 0]
[224, 285, 22, 714, 1, 0, 0, 0]
[258, 262, 42, 681, 0, 0, 0, 0]
[274, 247, 16, 565, 0, 0, 0, 0]
[301, 209, 6, 743, 0, 0, 0, 0]
[285, 276, 14, 623, 0, 0, -1, 1]
[253, 177, 24, 826, 0, 0, 0, 0]
[278, 254, 18, 590, 0, 1, 0, 0]
[257, 283, 38, 686, 0, 0, -1, 1]
[235, 266, 42, 716, 1, 1, 0, 0]
[275, 252, 10, 566, 0, 1, 0, 0]
[280, 269, 20, 651, 0, 0, 0, 0]
[298, 274, 40, 627, 0, 0, 1, 0]
[281, 220, 12, 744, 0, 0, 1, 0]
[258, 171, 28, 827, 0, 0, 2, 0]
[311, 253, 14, 591, 0, 0, 0, 0]
[257, 282, 42, 687, 0, 1, -1, 1]
[251, 285, 6, 718, 0, 1, 0, 0]
[275, 251, 39, 567, 0, 0, 0, 0]
[291, 281, 42, 628, 0, 0, 0, 0]
[284, 282, 30, 652, 0, 0, 0, 0][268, 223, 10, 748, 0, 0, 0, 0]

[251, 179, 32, 828, 0, 0, 2, 0]
[313, 256, 18, 592, 0, 1, 2, 0]
[258, 282, 35, 688, 0, 0, 0, 0]
[253, 284, 9, 719, 0, 0, -1, 1]
[287, 233, 6, 750, 0, 1, 0, 0]
[277, 251, 37, 568, 0, 0, 0, 0]
[296, 283, 39, 629, 0, 0, 1, 0]
[257, 162, 40, 831, 1, 0, 0, 0]
[301, 26

[207, 186, 10, 987, 0, 0, -1, 1][245, 207, 18, 959, 0, 0, -1, 1]

[239, 225, 20, 1064, 0, 1, 1, 0]
[274, 196, 36, 899, 1, 0, 0, 0]
[258, 251, 9, 928, 0, 1, -1, 1]
[190, 253, 42, 1022, 0, 1, -1, 1]
[228, 215, 40, 960, 0, 1, 0, 0]
[242, 204, 42, 1065, 0, 0, 0, 0]
[227, 171, 30, 990, 0, 0, -1, 1]
[203, 264, 38, 1026, 1, 1, 0, 0]
[232, 166, 26, 991, 0, 1, -1, 1]
[285, 202, 18, 1066, 0, 0, 0, 0]
[306, 276, 34, 1068, 0, 0, 0, 0]
VGlut1


In [27]:
outArray.shape

(813, 180)

In [11]:
#np.savetxt("rorb_avg_at.csv", np.column_stack([ids, outArray]), delimiter=",", fmt = "%d", header = "ids, ")
#np.concatenate((trmp, outArray), axis = 2)

In [12]:
#np.hstack((np.transpose(np.asarray(ids)),outArray))

In [166]:
if False:
    #head = "DAPI1, DAPI2, DAPI3, GABA, GAD2, Gephyrin, GluN1, MBP, PSD95, synapsin, TdTomato, VGlut1"
    #head = ['a' + str(i+1) + 'b' str(j+1) for i in range(len(u)) for j in range(len(CHAN_NAMES))]
    np.savetxt("rorb_annuli_stats_GoogleDoc.csv", outArray, delimiter=",", fmt = "%f")


In [14]:
if False:
    np.savetxt("rorb_gaussianAvg_at_orderIDS.csv", np.transpose(ids), delimiter=",", fmt = "%d", header = 'ids')

In [15]:
if False:
    np.savetxt("rorb_gaussianAvg_at_orderLocations.csv", loc, delimiter=",", fmt = "%d", header = 'x,y,z')

In [56]:
if LOCATIONS == "GoogleDocData.csv":
    head = "DAPI1, DAPI2, DAPI3, GABA, GAD2, Gephyrin, GluN1, MBP, PSD95, synapsin, TdTomato, VGlut1"
    np.savetxt("rorb_gaussianAvg_at_GoogleDoc.csv", np.transpose(outArray), delimiter=",", fmt = "%f", header = head)

In [34]:
if False:
    #head = "DAPI1, DAPI2, DAPI3, GABA, GAD2, Gephyrin, GluN1, MBP, PSD95, synapsin, TdTomato, VGlut1"
    #head = ['a' + str(i+1) + 'b' str(j+1) for i in range(len(u)) for j in range(len(CHAN_NAMES))]
    np.savetxt("rorb_annuli_stats_GoogleDocInf.csv", outArray, delimiter=",", fmt = "%f")

In [28]:
if True:
    #head = "DAPI1, DAPI2, DAPI3, GABA, GAD2, Gephyrin, GluN1, MBP, PSD95, synapsin, TdTomato, VGlut1"
    #head = ['a' + str(i+1) + 'b' str(j+1) for i in range(len(u)) for j in range(len(CHAN_NAMES))]
    np.savetxt("rorb_annuli_stats_GoogleDocL1W.csv", outArray, delimiter=",", fmt = "%f")